# Práctica 1.2 - CNNs (2024-2025) - Aprendizaje Profundo (Grado en IA)

|Integrantes|Correo electrónico|
|-----------|------------------|
|Hugo Fole Abellás|hugo.fole.abellas@udc.es|
|José Romero Conde|j.rconde@udc.es|

## REDES CONVOLUCIONALES
En esta segunda parte de la práctica desarrollaremos una red convolucional
(CNN) para resolver el mismo problema que en la primera parte, identificar el
elemento o animal que aparece en una fotografía.

Como ya comentamos, volveremos a utilizar el dataset `CIFAR10` de la librería `keras`. Dicho dataset contiene 60.000 imágenes a color de tamaño 32×32 de las que 50.000 se usarán para el entrenamiento de la red y 10.000 para testearla.

Las imágenes pertenecen a las 10 posibles categorı́as (6.000 imágenes por categorı́a).

Inicialmente, importaremos las librería a utilizar.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

2024-11-06 16:20:34.830853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730906434.844112    5601 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730906434.847847    5601 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 16:20:34.867101: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/folengui/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Un

A continuación, importaremos el dataset que será usado para entrenar la red que crearemos más adelante. 

In [2]:
from keras.datasets import cifar10

(x_train, y_train) , (x_test, y_test) = cifar10.load_data()

## NORMALIZACIÓN DE LOS DATOS
Para poder trabajar de manera correcta con este dataset, deberemos seguir ciertos procesos de normalización de datos, de manera que, o bien optimicemos su tiempo de ejecución o bien nos permita trabajar con el dataset. Estos procesos son :

    1. Realizar One-hot encoding en los targets.
    2. Normalizar los valores de las imágenes a float, ya que vienen en valores de 0 a 255.

Al utilizar redes convolucionales no es necesario aplanar las imágenes en vectores dado que estas realizan la dependencia entre capas utilizando más de un pixel.

### 1. One-hot encoding
Como ya comentamos, haremos one-hot en los targets de nuestro dataset ya que vienen divididos en 10 tipos de salidas categóricas representadas por números. Las diferentes salidas las podemos ver en la siguiente tabla.
|Número|Categoría|
|------|---------|
|0|airplane|
|1|automobile|
|2|bird|
|3|cat|
|4|deer|
|5|dog|
|6|frog|
|7|horse|
|8|ship|
|9|truck|

In [3]:
y_train = keras.utils.to_categorical(y_train, num_classes = 10)
y_test = keras.utils.to_categorical(y_test, num_classes = 10)

### 2.Normalización datos
Como ya dijimos, en este apartado realizaremos la parte de normalización de datos de **uint8** con valores entre [0-255] a **float64**, para que pasen a estar en el rango de valores [0,1] ya que esto nos permite trabajar con redes de neuronas, agiliza el entrenamiento de dichas redes y aumenta la precisión de estas.

In [4]:
x_train = x_train / 255.0
x_test = x_test / 255.0

## CREACIÓN REDES NEURONALES CONVOLUCIONALES 

Las redes neuronales convolucionales son un tipo de red que usa datos tridimensionales para la clasificación de imágenes. Se distinguen de otras redes por su alto rendimiento superior con entradas de imagen, voz o señales de audio. Para ello, tienen una composición especial diferente a la de las redes neuronales convencionales. Se componen de:

- `Capa convolucional` : suelen tener más de una y está compuesta por diversos filtros que tienen parámetros entrenables.
- `Capa de agrupación (pooling)` : estas capas se encargan de reducir la dimensionalidad de las capas, esto lo pueden hacer de dos maneras, por medio del valor máximo o del valor medio.
- `Capa "fully connected"` : esta última capa tendrá dimensión K, siendo K el número de clases en las que se puede clasificar. Como su nombre indica es una capa densa, es decir está completamente conectada. Utiliza una función **softmax** para proporcionar el resultado.

Comenzaremos creando una red sencilla para ver el funcionamiento y composición de esta.

In [5]:
from keras import layers
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)


# Classification part
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
#Compile
model.compile(optimizer="adam",
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()])
#Fitting
model.fit(x_train, y_train, epochs=10, batch_size=64)

I0000 00:00:1730895220.029954    4566 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2281 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
W0000 00:00:1730895220.209283    4758 gpu_backend_lib.cc:579] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda
  /home/folengui/.local/lib/python3.10/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/folengui/.local/lib/python3.10/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  /home/folengui/.local/lib/python3.10/site-packages/tensorflow/python/platform/../../cuda
  .
You can choose the search directory by setting xla_gpu_cuda_data_d

InternalError: {{function_node __wrapped__FloorMod_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:FloorMod] name: 

In [ ]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {acc:.3f}")

In [7]:
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 341,216 (1.30 MB)

 Trainable params: 113,738 (444.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 227,478 (888.59 KB)